Найти решение краевой задачи для одномерного стационарного уравнения теплопроводности
$$\frac{d}{dx}[k(x)\frac{du}{dx}] - q(x)u = -f(x)$$
в 11 равноудаленных точках отрезка $[0, 1]$ с относительной точностью $0.0001$.
Отладку программы произвести на модельной задаче с постоянными коэффициентами.

In [157]:
import numpy as np
from pandas import DataFrame

Условие. Задача 4.

Краевые условия:
$$u(0) = 1; u(1) = 0$$
\begin{equation*}
 \begin{cases}
    u(x_0 - 0) = u(x_0 + 0) \\
    k(x_0 - 0) u_{x}(x_0 - 0) = k(x_0 + 0) u_{x}(x_0 + 0) 
 \end{cases}
\end{equation*}
$$x < x_0 = 1/\sqrt{2} : k(x) = e^{-x}; q(x) = x^2; f(x) = 1$$
$$x > x_0: k(x) = 1; q(x) = e^{-x^2}; f(x) = cos(x)$$

Модельная задача:
$$x_0 = 1 / \sqrt{2} : k(x) = k(x_0); q(x) = q(x_0); f(x) = f(x_0)$$

In [158]:
def k(t : float, t0: float, flag = 0) -> float:
    if t < t0:
        return np.exp(-t)
    elif t > t0:
        return 1
    else:
        if flag == 0:
            return np.exp(-t)
        else: 
            return 1


def q(t : float, t0 : float, flag = 0) -> float:
    if t < t0:
        return t**2
    elif t > t0:
        return np.exp(-(t)**2)
    else:
        if flag == 0:
            return t**2
        else: 
            return np.exp(-(t)**2)

def f(t : float, t0 : float, flag = 0) -> float:
    if t < t0:
        return 1
    elif t > t0:
        return np.cos(t)
    else:
        if flag == 0:
            return 1
        else: 
            return np.cos(t)

In [159]:
#условия
accur = 10**(-4)

u0 = 1
u1 = 0

x_0 = 2**(-0.5)

k1 = k(x_0, x_0)
k2 = k(x_0, x_0, 1)

q1 = q(x_0, x_0)
q2 = q(x_0, x_0, 1)

f1 = f(x_0, x_0)
f2 = f(x_0, x_0, 1)

#разделения отрезка на n частей
L = 11

x = np.round(np.linspace(0.0, 1.0, L), 7)



Аналитическое решение модельной задачи

In [160]:
#решение характерестического уравнения
lambda1 = (q1 / k1)**0.5
lambda2 = (q2 / k2)**0.5

#частные решения
mu1 = f1 / q1
mu2 = f2 / q2

#вспомогательные коэффициенты
a11 = np.exp(-lambda1 * x_0) - np.exp(lambda1 * x_0)
a12 = np.exp(lambda2 * (2 - x_0)) - np.exp(lambda2 * x_0)
a21 = k1 * lambda1 * (np.exp(-lambda1 * x_0) + np.exp(lambda1 * x_0))
a22 = k2 * lambda2 * (np.exp(lambda2 * (2 - x_0)) + np.exp(lambda2 * x_0)) 

b1 = mu2 - mu1 + (mu1 - u0) * np.exp(lambda1 * x_0) - (mu2 - u1) * np.exp(lambda2 * (1 - x_0))
b2 = k1 * lambda1 * (u0 - mu1) * np.exp(lambda1 * x_0) + k2 * lambda2 * (u1 - mu2) * np.exp(lambda2 * (1 - x_0))

#коэффициенты для общего решения
c1 = (((u0 - mu1) * a11 - b1) * a22 - ((u0 - mu1) * a21 - b2) * a12) / (a11 * a22 - a12 * a21) 
c2 = (b1 * a22 - b2 * a12) / (a11 * a22 - a12 * a21)
c3 = (b2 * a11 - b1 * a21) / (a11 * a22 - a12 * a21)
c4 = (u1 - mu2) * np.exp(lambda2) - c3 * np.exp(2 * lambda2)

#нахождение аналитического решения
u_alpha = np.round(c1 * np.exp(lambda1 * x[x<x_0]) + c2 * np.exp(-lambda1 * x[x<x_0]) + mu1, 7)
u_beta = np.round(c3 * np.exp(lambda2 * x[x>x_0]) + c3 * np.exp(lambda2 * x[x>x_0]) + mu2, 7)

u_analytical = list(u_alpha) + list(u_beta)


Численное решение модельной задачи с заданной степенью точности(Метод встречных прогонок)

In [161]:
h = 1 / L
l_1 = np.max(np.where(x <= x_0))
l_2 = np.min(np.where(x >= x_0))

# e = l = [1, l_1 -1]
ae = k1
be = -2 * k1 - q1 * h**2 
ce = k1
de = -f1 * h**2

#r = l = [L - 1; l_2 + 1]
ar = k2
br = -2 * k2 - q2 * h**2
cr = k2
dr = -f2 * h**2

#прямая прогонка
alpha1 = -ae / be
beta1 = (de - ce * u0) / be

alphaL = -cr / br 
betaL = (dr - cr * u1) / br

coefs1 = [[alpha1, beta1]]
coefs2 = [[alphaL, betaL]]

for i in range(1, l_1 - 1):
    coefs1.append([- ae / (be + ce * coefs1[i - 1][0]), (de - ce * coefs1[i-1][1]) / (be + ce * coefs1[i-1][0])])

for i in range(1, L - l_2 - 1):
    coefs2.append([- cr / (br + ar * coefs2[i - 1][0]), (dr - ar * coefs2[i - 1][1]) / (br + ar * coefs2[i - 1][0])])

u_l_ab = (k1 * coefs1[-1][1] + k2 * coefs2[0][1]) / (k1 * (1 - coefs1[-1][0]) + k2 * (1 - coefs2[0][0]))
print(u_l_ab)
#обратная прогонка
coefs1.reverse()
u_a = [coefs1[0][0] * u_l_ab + coefs1[0][1]]
for i in range(1, l_1):
    u_a.append(coefs1[i][0] * u_a[i - 1] + coefs1[i][1])
u_a.reverse()

u_b = [coefs2[0][0] * u_l_ab + coefs2[0][1]]
for i in range(1, L - l_2 + 1):
    u_b.append(coefs2[i][0] * u_b[i - 1] + coefs2[i][1])

u_sweep = u_a + u_b


0.1604692530840921


IndexError: list index out of range

In [ ]:
print(*x)
print('Analytical solve:', *u_analytical)
print('Sweep solve:', *u_sweep)
print('Error:', *(np.array(u_analytical) - np.array(u_sweep)))
print('Max error:', max(np.array(u_analytical) - np.array(u_sweep)))

0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0
Analytical solve: 1.0 0.9322537 0.8536708 0.7634536 0.6606864 0.5443264 0.4131926 0.265954 -0.8888377 -1.0623466 -1.2499084
Sweep solve: 0.9387626294404172 0.8686314100778756 0.7890185973806791 0.6992569849927854 0.5985943131172495 0.48618696409736284 0.36109289236042086 0.11398768115619673 0.08192750751395507 0.07043541944770111 0.06833984774539287
Error: 0.06123737055958278 0.06362228992212438 0.06465220261932081 0.06419661500721452 0.062092086882750475 0.05813943590263715 0.05209970763957916 0.1519663188438033 -0.970765207513955 -1.132782019447701 -1.318248247745393
Max error: 0.1519663188438033


Численное решение задачи с переменными коэффициентами